# 1. Setting up the environment
 The "pip install -e ./" will look for the setup.py in root directory and install "st_water_seg" as a package. what it oes is it will look for the "__init__.py" under "st_water_seg" and their subfolers, can make them a package. They together become the st_water_seg package

# 2.Need Fix
1. 记得看ignore index
2. predict中的读取模型也需要直接读取class，而不是instance
3. 标记resample input image
4. use pathlib so that both windows and linux can use this code
4. 传入参数
5. 传入文件保存路径之类的
6.记得load_from_checkpoint需要直接用class类，不能是instance
getcwd检测的是你在哪执行的代码

# How code works
这个代码的核心读取是围绕crop进行展开的。在创建所有数据集的时候，都必须产生crop_slices. 注意的是每个crop_slice都是针对一个输入图像产生的。所以这个每个图片不会混在一起，但是边角会不会加0值或者nan呢？这个是通过dataset——utils中的get_crop_slices功能实现的。
在预测产生prediction或者inference的时候使用image_sticher来使用不同的策略去拼接这些预测的结果。会将读取的一个region的所有图像先拼接在一起，然后统一进行格式转换。最后一个文件夹的所有东西统一保存。在统一转换保存的时候会需要非常多的内存。
## 数据集构建 _prepare_data,这个是在构建类的时候直接调用执行的，然后会根据sensor与输入channel的不同，去获取n_channels的数量（int）
先读取数据路径, 
## get_crop_slices
输入是标签的里面返回的值存的是各个slice的coordinate。
一般用的是exact模式：
我自己将exact修改后变成了exact_improve模式

先判断能完整的切多少，将完整切片的坐标加入list
再计算剩余的边角，先计算宽度方向的，再计算长度方向的，最后计算最右下角的坐标，这些统一加入到list中

# 读取并准备data-loader
1.先是数据集构建，_prepare_data,这个是在构建类的时候直接调用执行的，然后会根据sensor与输入channel的不同，去获取n_channels的数量（int）
## 1. 读取文件的时候，先通过 _prepare_data这个func来准备输入数据与dictionary
 1. 根据root_dir加上“CSDA”这个文件夹名，来glob所有region的路径，然后获取所有region的名字
 2. 然后以region_name为检索值，创建region_dir的字典（ construct a dictionary that maps region names to their corresponding directory paths）
    具体是使用双元素for循环以及zip来进行concurrent循环， “a for loop that iterates over two lists simultaneously: region_names and region_dirs. The zip function is used to aggregate elements from these two lists into pairs, making it possible to iterate over both lists in parallel. For each iteration, region_name will be the current element from the region_names list, and region_dir will be the corresponding element from the region_dirs list.”
## 2. 建好字典之后，将这个dictionary与sensor_name一起输入_split_data(region_dirs_dict, sensor_name)这个函数中，获取以不同方式group之后的image_paths
    1. 先看传进来的是不是一个空dict
    2. 然后根据split模式调整dict中的region
        1.然后看设置的eval_region是不是none，如果是的话，将所有数据读入
        2. 然后如果split模式设置为train的话，将eval_region从region_dirs这个dict中删除
        3. 如果split模式为valid或者test的话：将eval_region这个key单独提出来，单独新建一个dict，命名为sub_region_dirs
        4. 然后根据